### The Data

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/stars.csv')

In [3]:
df.sample(5)

,description,language,repo,stargazers,user
12680,Just draw a bounding box and you can remove th...,Python,zllrunning/video-object-removal,1390,greed2411
2111,NaN,Python,jmcarp/robobrowser,3369,stribny
8272,:mailbox_with_mail: Easily and securely share ...,Rust,timvisee/ffsend,4116,mraza007
6675,Header-only C++ binding for libzmq,C++,zeromq/cppzmq,684,jrezzende
8457,An Implementation of Model-Agnostic Meta-Learn...,Python,tristandeleu/pytorch-maml,26,fly51fly


### The Cleanup

In [4]:
df = df[df['repo'] != 'maxhumber/gazpacho']
df = df[df.language.isin(['Python', 'Jupyter Notebook'])]
popular = pd.DataFrame(df['repo'].value_counts())
select_repos = popular[popular['repo'] >= 5].index.tolist()
df = df[df['repo'].isin(select_repos)]

In [5]:
df.sample(5)

,description,language,repo,stargazers,user
2812,Static site generator that supports Markdown a...,Python,getpelican/pelican,9123,javad94
314,A next generation HTTP client for Python. 🦋,Python,encode/httpx,1928,robsalasco
4638,A simple Flask boilerplate app with SQLAlchemy...,Python,hack4impact/flask-base,2108,fahmihidayah
16123,A course that helps newbies master the core pr...,Python,purcellconsult/Cracking-Python-Bootcamp,254,dsfb
23065,Get and set values in your .env file in local ...,Python,theskumar/python-dotenv,2508,malfario


### The Preparation

In [6]:
df = df.groupby(['user'])['repo'].apply(lambda x: ','.join(x))
df = pd.DataFrame(df)

In [7]:
df.sample(5)

,repo
user,
dinko-pehar,"marshmallow-code/marshmallow,flask-restful/fla..."
CalebWhitehead,"chubin/wttr.in,grapheneX/grapheneX,psf/request..."
RodolfoFerro,"arraiyopensource/kornia,plainas/flask-swagger-..."
dillionaire,sloria/TextBlob
03b8,"python/cpython,hugapi/hug,HypothesisWorks/hypo..."


### The Model

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors


class NNRecommender:
    def __init__(
        self, n_neighbors=10, max_features=1000, tokenizer=lambda x: x.split(",")
    ):
        self.cv = CountVectorizer(tokenizer=tokenizer, max_features=max_features)
        self.nn = NearestNeighbors(n_neighbors=n_neighbors)

    def fit(self, X):
        self.X = X
        X = self.cv.fit_transform(X)
        self.nn.fit(X)
        return self

    def predict(self, X):
        Xp = []
        for Xi in X:
            Xt = self.cv.transform([Xi])
            _, neighbors = self.nn.kneighbors(Xt)
            repos = []
            for n in neighbors[0]:
                r = self.X.iloc[int(n)].split(",")
                repos.extend(r)
            repos = list(set(repos))
            repos = [r for r in repos if r not in Xi.split(",")]
            Xp.append(repos)
        return Xp

In [9]:
model = NNRecommender()
model.fit(df['repo'])

In [10]:
df['repo'][42]

'CorentinJ/Real-Time-Voice-Cloning,s0md3v/goop,psf/requests,slundberg/shap,facebookresearch/pytext,practicalAI/practicalAI,Zulko/moviepy,python/mypy,ParthS007/background,keon/algorithms,google/python-fire,eriklindernoren/ML-From-Scratch,minimaxir/big-list-of-naughty-strings,pudo/dataset,nvbn/thefuck,dbcli/pgcli,faif/python-patterns,chriskiehl/Gooey'

In [11]:
model.predict([df['repo'][42]])

[['smacke/subsync',
  'dae/anki',
  'deepfakes/faceswap',
  'fastai/fastai',
  'sherlock-project/sherlock',
  'ageron/handson-ml2',
  'donnemartin/interactive-coding-challenges',
  'weskerfoot/DeleteFB',
  'shengqiangzhang/examples-of-web-crawlers',
  'scikit-learn/scikit-learn',
  'Miserlou/Zappa',
  'sloria/TextBlob']]

### Under the Hood

In [12]:
cv = CountVectorizer(tokenizer=lambda x: x.split(','), max_features=1000)
X = cv.fit_transform(df['repo'])
X.todense()[:5]

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 1, 1, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [13]:
nn = NearestNeighbors(n_neighbors=10)

In [14]:
nn.fit(X)
dist, ind = nn.kneighbors()
ind[:5]

array([[106,  14,  99,  48,  65,   3,  10,  19,   2,  40],
       [ 14,  66,  99,  10,  19,  65,  48,   3,  40,  12],
       [ 10,  19,   3,  99,  48,  14,  65,  40,  66,  92],
       [ 48,  10,  99,  19,  14,  65,   2,  40,  92,  72],
       [ 28,  10,  48,  72,  20,  19,  99,  14,  65,   3]])

#### Pickle 

In [15]:
import pickle

In [16]:
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

PicklingError: Can't pickle <function NNRecommender.<lambda> at 0x121c6c048>: attribute lookup NNRecommender.<lambda> on __main__ failed

In [17]:
import dill # pip install dill

with open('model.pkl', 'wb') as f:
    dill.dump(model, f)

del model

with open('model.pkl', 'rb') as f:
    model = dill.load(f)

In [19]:
model.predict(['streamlit/streamlit,huggingface/transformers,encode/httpx,aws/chalice,maxhumber/chart'])

[['mherrmann/fbs',
  'minimaxir/big-list-of-naughty-strings',
  'TheAlgorithms/Python',
  'shengqiangzhang/examples-of-web-crawlers',
  'ytdl-org/youtube-dl',
  'Avik-Jain/100-Days-Of-ML-Code',
  'plasticityai/supersqlite',
  'scikit-learn/scikit-learn',
  'sloria/TextBlob',
  'PySimpleGUI/PySimpleGUI',
  'vinta/awesome-python',
  'fastai/fastai',
  'donnemartin/system-design-primer']]